In [6]:
from PIL import Image
import rasterio
from rasterio.mask import mask
import pyproj
import shapely
import shapely.ops
import numpy as np
import glob
import matplotlib.pyplot as plt
import os
from shapely.geometry import box, Polygon, MultiPolygon

In [2]:
# fp = '/oak/stanford/groups/deho/building_compliance/san_jose_naip/2020/'
# fp_out = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/'

fp = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint/'
fp_out = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512/'

## Preprocess: move TIF to parent directory

Only for LA

In [ ]:
from os import listdir, rmdir
from shutil import move
for i in glob.glob(f'{fp}*'):
    for j in glob.glob(f'{i}/*'):
        for filename in listdir(j):
            move(os.path.join(j, filename), os.path.join(i, filename))
        rmdir(j)

## Take raw NAIP TIFs and crop according to bounding box specified
- this avoids having incomplete building TIFs

In [19]:
# /oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint/wilshire/m_3411861_se_11_h_20160711.tif
fp = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint/'
if not os.path.exists(os.path.join(fp, 'cropped')):
    os.mkdir(os.path.join(fp, 'cropped'))

In [20]:

south_la = box(-118.36618406252838, 33.92399439434224, -118.24340101311658, 34.01043519525057)
mid_wilshire = box(-118.42264979853323, 34.020425261173386, -118.3274039386137, 34.051457250941674)
torrance = box(-118.38894671860481, 33.76190036981278, -118.26456955803374, 33.837507964798)
valley = box(-118.5947617037504, 34.1518934898276, -118.40286108520485, 34.278004449211494)

In [37]:
dic = {'wilshire': mid_wilshire, 'south_la': south_la, 'torrance': torrance, 'valley': valley}
for k, v in dic.items():
    for i in os.listdir(os.path.join(fp, k)):
        if 'tif' in i:
            print(i)
            with rasterio.open(os.path.join(fp, k, i)) as inds:
                bounds = inds.bounds
                geom = box(*bounds)
                wgs84 = pyproj.CRS('EPSG:26911')
                utm = pyproj.CRS('EPSG:4326')

                project = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform
                utm_point = shapely.ops.transform(project, v)
                
#                 print(geom)
                
                # when you transform a box, the coordinate system changes such that it doesn't become a box anymore.
                # the result is a slanted TIF, which we don't want because it will leave black pixels on the edges
                coords_x, coords_y = utm_point.exterior.coords.xy
                #box is minx, miny, maxx, maxy
                new_box = box(max(coords_x[2], coords_x[3]), max(coords_y[3], coords_y[4]),
                    min(coords_x[0], coords_x[1]), min(coords_y[1], coords_y[2]))

                mask_im, trans = rasterio.mask.mask(inds, [geom.intersection(new_box)], crop=True, filled=True)
#                 print(mask_im.shape)
                
#                 mask_im = np.rollaxis(mask_im, 0, 3)
#                 plt.imshow(mask_im[:, :, :3])
#                 plt.show()
                
                out_meta = inds.meta.copy()
                out_meta.update({'width': mask_im.shape[2],
                                'height': mask_im.shape[1],
                                'transform': trans})
                
                with rasterio.open(os.path.join(fp, 'cropped', i), 'w', **out_meta) as outds:
                    outds.write(mask_im)

m_3411862_sw_11_h_20160710.tif
m_3411861_se_11_h_20160711.tif
m_3411862_sw_11_h_20160710.tif
m_3411862_se_11_h_20160707.tif
m_3311807_nw_11_h_20160706.tif
m_3311806_nw_11_h_20160710.tif
m_3311806_sw_11_h_20160710.tif
m_3311806_se_11_h_20160707.tif
m_3311806_ne_11_h_20160707.tif
m_3411863_sw_11_h_20160706.tif
m_3311807_sw_11_h_20160706.tif
m_3311814_se_11_h_20160707.tif
m_3311814_ne_11_h_20160707.tif
m_3311813_ne_11_h_20160711.tif
m_3311814_nw_11_h_20160710.tif
m_3311813_se_11_h_20160711.tif
m_3311814_sw_11_h_20160710.tif
m_3411844_se_11_h_20160616.tif
m_3411852_ne_11_h_20160616.tif
m_3411852_sw_11_h_20160616.tif
m_3411853_ne_11_h_20160711.tif
m_3411853_sw_11_h_20160711.tif
m_3411844_sw_11_h_20160616.tif
m_3411852_nw_11_h_20160616.tif
m_3411845_se_11_h_20160711.tif
m_3411853_nw_11_h_20160711.tif
m_3411853_se_11_h_20160711.tif
m_3411852_se_11_h_20160616.tif
m_3411845_sw_11_h_20160711.tif


In [8]:
fp = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512/raw_tif/'

for i in os.listdir(os.path.join(fp)):
    with rasterio.open(os.path.join(fp, i)) as inds:
        # get TIF bounds
        bounds = inds.bounds
        geom = box(*bounds)
        
        # prepare to convert TIF bounds to standard 4326
        wgs84 = pyproj.CRS('EPSG:26911')
        utm = pyproj.CRS('EPSG:4326')

        project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
        
        # convert
        utm_geom = shapely.ops.transform(project, geom)
        print(utm_geom)

POLYGON ((-118.2983176298719 33.9876840237844, -118.2983597613052 33.99045392746032, -118.3016850831615 33.99041878583038, -118.3016428438966 33.98764888579832, -118.2983176298719 33.9876840237844))
POLYGON ((-118.2742226555658 33.99837889825981, -118.2742325118646 33.99903891959068, -118.2775581997981 33.99900442242915, -118.2775483177918 33.99834440195053, -118.2742226555658 33.99837889825981))
POLYGON ((-118.4230942914684 34.15856854141169, -118.4231407685279 34.16133822421479, -118.426472601782 34.16129954102149, -118.426426016001 34.15852986221945, -118.4230942914684 34.15856854141169))
POLYGON ((-118.4425472195135 34.17572016682343, -118.4425943620732 34.17848981827915, -118.445926840913 34.17845058984118, -118.4458795895435 34.17568094244191, -118.4425472195135 34.17572016682343))
POLYGON ((-118.5706673011233 34.23826036556976, -118.5707187501318 34.24102982534232, -118.5740534944384 34.24098704945408, -118.5740019363099 34.23821759410064, -118.5706673011233 34.23826036556976))


POLYGON ((-118.3256227280237 33.96870384120982, -118.3256657146812 33.97147372311313, -118.3289902610938 33.97143786101982, -118.3289471667103 33.96866798283605, -118.3256227280237 33.96870384120982))
POLYGON ((-118.5151034100192 34.17560312566203, -118.5151529229693 34.17837268633757, -118.5184852877474 34.17833148766351, -118.5184356660024 34.175561931248, -118.5151034100192 34.17560312566203))
POLYGON ((-118.4723735643454 34.22954672106385, -118.4724217786106 34.2323163113813, -118.4757563320393 34.2322762179083, -118.4757080086811 34.22950663173354, -118.4723735643454 34.22954672106385))
POLYGON ((-118.304714760922 33.7835057035639, -118.304756775645 33.78627569211503, -118.3080741758291 33.78624056339928, -118.308032054356 33.78347057850152, -118.304714760922 33.7835057035639))
POLYGON ((-118.5052541814008 34.18403488253023, -118.5053033881116 34.18680445198622, -118.5086360996083 34.18676351195776, -118.5085867840554 34.18399394673457, -118.5052541814008 34.18403488253023))
POLYG

POLYGON ((-118.3702248239648 33.76210945156254, -118.3702557368176 33.76405164449228, -118.3735721915455 33.76401477614343, -118.373541203931 33.76207258590303, -118.3702248239648 33.76210945156254))
POLYGON ((-118.3416110201205 33.92814409220573, -118.3416544588117 33.93091397433352, -118.3449774067101 33.93087771825333, -118.3449338605109 33.92810783988813, -118.3416110201205 33.92814409220573))
POLYGON ((-118.4958614893992 34.23757287206006, -118.4959104873608 34.24034242923774, -118.4992453216437 34.24030168870734, -118.4991962145506 34.23753213573867, -118.4958614893992 34.23757287206006))
POLYGON ((-118.2681435990134 33.97137657967168, -118.2681847262075 33.97414652348088, -118.271509452521 33.9741122118028, -118.2714682175768 33.97134227155229, -118.2681435990134 33.97137657967168))
POLYGON ((-118.2647439170842 33.77284011117166, -118.2647846284886 33.77561014799568, -118.2681016696627 33.77557610335068, -118.2680608515576 33.7728060700679, -118.2647439170842 33.77284011117166))

POLYGON ((-118.3079057244672 33.77516061605714, -118.3079478286758 33.7779306048304, -118.3112649028993 33.77789539794657, -118.3112226919851 33.77512541283527, -118.3079057244672 33.77516061605714))
POLYGON ((-118.5241567182208 34.18817315985927, -118.5242065504431 34.19094270320658, -118.5275393954998 34.19090124537197, -118.527489454417 34.18813170631072, -118.5241567182208 34.18817315985927))
POLYGON ((-118.2984454655943 33.93626869108333, -118.298487519767 33.93903861779189, -118.3018108412438 33.93900351923026, -118.3017686795126 33.93623359616381, -118.2984454655943 33.93626869108333))
POLYGON ((-118.3110677782962 33.82871201642352, -118.3111100695757 33.83148197761016, -118.3144292064596 33.83144663670964, -118.3143868081928 33.828676679196, -118.3110677782962 33.82871201642352))
POLYGON ((-118.3780858043625 33.78129031625573, -118.37813017765 33.78406022247353, -118.3814473924494 33.78402312329361, -118.3814029124363 33.78125322093413, -118.3780858043625 33.78129031625573))
PO

POLYGON ((-118.561172772505 34.27055180250717, -118.5612239724821 34.27332126025555, -118.5645600063576 34.27327870761185, -118.5645086970847 34.27050925425753, -118.561172772505 34.27055180250717))
POLYGON ((-118.3507046703706 33.78643051493868, -118.3507481706015 33.78920045047612, -118.354065621554 33.78916408254622, -118.3540220145655 33.78639415079079, -118.3507046703706 33.78643051493868))
POLYGON ((-118.5675695626787 34.25107935257265, -118.5676209349088 34.25384881066086, -118.5709561898375 34.25380610507657, -118.5709047084178 34.2510366513994, -118.5675695626787 34.25107935257265))
POLYGON ((-118.5308221837954 34.18809016228381, -118.5308722337378 34.19085969704964, -118.534205065142 34.1908180582402, -118.534154906341 34.18804852777912, -118.5308221837954 34.18809016228381))
POLYGON ((-118.525000499378 34.16994017142616, -118.5250503251027 34.17270972192558, -118.5283824519406 34.17266826048962, -118.5283325174535 34.16989871427775, -118.525000499378 34.16994017142616))
POLY

...

POLYGON ((-118.3467891788483 33.81447500506483, -118.3468325988853 33.81724493246322, -118.3501511375096 33.81720864343907, -118.3501076105665 33.81443871981295, -118.3467891788483 33.81447500506483))
POLYGON ((-118.2984018984936 33.99322382984104, -118.298444041438 33.99599373092651, -118.3017695790019 33.99595858200817, -118.3017273281963 33.99318868456702, -118.2984018984936 33.99322382984104))
POLYGON ((-118.5602031130833 34.15368823430445, -118.5602540576259 34.15645774617721, -118.5624858582309 34.15642935173573, -118.562434840889 34.15365984279987, -118.5602031130833 34.15368823430445))
POLYGON ((-118.3304578142988 33.76448711679906, -118.3305006271391 33.7672570851808, -118.3338172598513 33.76722128202366, -118.3337743403655 33.76445131736642, -118.3304578142988 33.76448711679906))
POLYGON ((-118.3854275888077 33.82512851271476, -118.3854722722355 33.82789839063129, -118.3887911687627 33.82786105167982, -118.3887463783796 33.82509117764402, -118.3854275888077 33.82512851271476)

POLYGON ((-118.3294107399605 33.99844410533207, -118.3294109085338 33.99845492511677, -118.3327365005736 33.99841893686599, -118.3327363315788 33.99840811709586, -118.3294107399605 33.99844410533207))
POLYGON ((-118.3702998293538 33.76682156756829, -118.3703439279157 33.76959148934829, -118.3736605959306 33.76955461332783, -118.3736163907179 33.76678469538376, -118.3702998293538 33.76682156756829))
POLYGON ((-118.3543710373732 33.80855356856341, -118.3544146920567 33.81132348995507, -118.3577329916291 33.81128700252674, -118.3576892300722 33.80851708492828, -118.3543710373732 33.80855356856341))
POLYGON ((-118.3107251963365 33.92847676636364, -118.3107676357124 33.93124668301952, -118.3117476624415 33.93123624535266, -118.3117051913569 33.92846632977998, -118.3107251963365 33.92847676636364))
POLYGON ((-118.4359040812258 34.19320926632253, -118.4359510375153 34.19597891796606, -118.4392842139778 34.19593985248508, -118.4392371487835 34.19317020488015, -118.4359040812258 34.193209266322

POLYGON ((-118.5516304965931 34.23215098061061, -118.5516813105476 34.23492046830467, -118.5550158443108 34.23487821677359, -118.5549649212652 34.2321087334449, -118.5516304965931 34.23215098061061))
POLYGON ((-118.2785880681706 33.81148361113654, -118.2786292851498 33.81425361567955, -118.281947799222 33.81421916442008, -118.2819064753408 33.81144916345855, -118.2785880681706 33.81148361113654))
POLYGON ((-118.3107531326477 33.9937962478477, -118.3107956773296 33.99656613503336, -118.3127507338581 33.99654528561091, -118.3127081257651 33.99377540058694, -118.3107531326477 33.9937962478477))
POLYGON ((-118.3467310968656 33.81076914314098, -118.3467457647438 33.81170507637123, -118.3500640895706 33.81166879489136, -118.3500493855781 33.8107328629356, -118.3467310968656 33.81076914314098))
POLYGON ((-118.3491562086911 33.98507006209837, -118.3491999852772 33.98783990997799, -118.3525251368055 33.98780339676332, -118.3524812524105 33.98503355266985, -118.3491562086911 33.98507006209837))


POLYGON ((-118.5895524368861 34.17824288102715, -118.589604387338 34.18101234267557, -118.5929367380062 34.18096911939868, -118.5928846787606 34.17819966221931, -118.5895524368861 34.17824288102715))
POLYGON ((-118.4334152395339 34.24310183903825, -118.4334622021857 34.24587147113971, -118.4367973479056 34.24583242360544, -118.4367502760803 34.24306279553786, -118.4334152395339 34.24310183903825))
POLYGON ((-118.4191141860483 34.24811606098685, -118.4191477910524 34.25011756390496, -118.4224831251244 34.2500789015788, -118.4224494412053 34.24807740154692, -118.4191141860483 34.24811606098685))
POLYGON ((-118.5385446853815 34.24616657515548, -118.5385950973608 34.24893607362517, -118.5419302044812 34.24889416298647, -118.5418796833327 34.24612466884607, -118.5385446853815 34.24616657515548))
POLYGON ((-118.4949599008637 34.24921635120303, -118.4950088906661 34.25198590425452, -118.498344185603 34.25194517616129, -118.4982950866062 34.24917562731684, -118.4949599008637 34.24921635120303)

KeyboardInterrupt: 

# Begin splitting tile

In [38]:
fp = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint/cropped/'
fp_out = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512_v2/'

In [39]:
if not os.path.exists(fp_out):
    os.mkdir(fp_out)

if not os.path.exists(os.path.join(fp_out, 'raw_tif')):
    os.mkdir(os.path.join(fp_out, 'raw_tif'))

if not os.path.exists(os.path.join(fp_out, 'train')):
    os.mkdir(os.path.join(fp_out, 'train'))
    os.mkdir(os.path.join(fp_out, 'train', 'images'))
    os.mkdir(os.path.join(fp_out, 'train', 'masks'))

In [40]:
def save_tif_npy(numpyImage, fn_out):
    # save to both npy
    np.save(os.path.join(fp_out, 'train', 'images', f'{fn_out}.npy'),
            numpyImage)

In [ ]:
for i in glob.glob(f'{fp}*'):
    if 'tif' in i:
        tif_name = i.split('/')[-1].split('.')
        im = Image.open(i)
        numpyImage = np.array(im)
        row_min = 0
        row_max = 512
        count = 0
        while row_min < numpyImage.shape[0]:
            col_min = 0
            col_max = 512
            while col_min < numpyImage.shape[1]:
                fn_out = f'{tif_name[0]}_{count}'
                if row_max > numpyImage.shape[0]:
                    if col_max > numpyImage.shape[1]:
                        save_tif_npy(numpyImage[row_min:, col_min:, :], fn_out)
                        count += 1
                    else:
                        save_tif_npy(numpyImage[row_min:, col_min:col_max, :], fn_out)
                        count += 1
                else:
                    if col_max > numpyImage.shape[1]:
                        save_tif_npy(numpyImage[row_min:row_max, col_min:, :], fn_out)
                        count += 1
                    else:
                        save_tif_npy(numpyImage[row_min:row_max, col_min:col_max, :], fn_out)
                        count += 1
                col_min = col_max
                col_max += 512
            row_min = row_max
            row_max += 512

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (103529670 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (125518800 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (125210400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (125294000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (98718250 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS a

In [ ]:
# SPLIT TIF
import os
from itertools import product
from rasterio import windows

in_path = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint/cropped/'
out_path = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512_v2/raw_tif/'

def get_tiles(ds, width=512, height=512):
    nols, nrows = ds.meta['width'], ds.meta['height']
#     print(nols, nrows)
    offsets = product(range(0, nrows, height), range(0, nols, width))
    offsets = [(i[1], i[0]) for i in offsets]
#     print(offsets)
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform

for i in glob.glob(f'{in_path}*'):
    if 'tif' in i:
        input_filename = i.split('/')[-1].split('.')
        fn_out = '{}_{}.tif'

        with rasterio.open(i) as inds:
            tile_width, tile_height = 512, 512

            meta = inds.meta.copy()

            count = 0
            for window, transform in get_tiles(inds):
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path,fn_out.format(input_filename[0], count))
                with rasterio.open(outpath, 'w', **meta) as outds:
                    outds.write(inds.read(window=window))
                count += 1

## Generate masks from OSMBuildings

In [3]:
# fp = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/raw_tif/'
# fp_out = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/mask/'

fp = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512_v2/raw_tif/'
fp_out = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512_v2/'

In [4]:
from shapely.geometry import box
import pyproj
from shapely.ops import transform
import geopandas as gpd
from shapely.ops import unary_union
import shapely
from rasterio.mask import mask
import pickle

Open OSMBuildings shapefile

In [5]:
len(os.listdir(fp))

5897

In [6]:
no_buildings = []
if os.path.exists('data/los_angeles/no_buildings_v2.txt'):
    with open('data/los_angeles/no_buildings_v2.txt', 'r') as f:
        no_buildings = f.readlines()
        no_buildings = [line.rstrip() for line in no_buildings]

done = os.listdir(os.path.join(fp_out, 'train', 'masks'))

In [11]:
import multiprocessing as mp

all_files = glob.glob(f'{fp}*')

In [12]:
def driver(i):
    fn = i.split('/')[-1].split('.')[0]
    if fn in no_buildings_global or f'{fn}_mask.npy' in done_global:
        pass
    else:
        with rasterio.open(i) as inds:
            bounds = inds.bounds
            geom = box(*bounds)
            wgs84 = pyproj.CRS('EPSG:26911')
            utm = pyproj.CRS('EPSG:4326')

            project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
            utm_point = shapely.ops.transform(project, geom)
            
            # MOVED TO OAK
#             df = gpd.read_file('data/los_angeles/la_select_footprints.geojson', mask=utm_point)
            df = gpd.read_file('/oak/stanford/groups/deho/building_compliance/los_angeles_suppl/la_select_footprints.geojson', 
                               mask=utm_point)        
    if len(df) == 0:
                with open('data/los_angeles/no_buildings_v2.txt', 'a') as f:
                    f.write(f'{fn}\n')

            else:
                df = df.to_crs('epsg:26911')
                mask_im, trans = rasterio.mask.mask(inds, list(df['geometry']))
                mask_im = np.rollaxis(mask_im, 0, 3)

                mask = np.zeros((mask_im.shape[0], mask_im.shape[1]))
                mask[np.sum(mask_im == 0, axis=2) < 4] = 1 # need to check that this is right

                np.save(os.path.join('/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512_v2/train/masks/',
                                     f'{fn}_mask.npy'),
                    mask)

                
def make_global(no_buildings, done):
    global no_buildings_global
    global done_global
    
    no_buildings_global = no_buildings
    done_global = done


In [13]:
nprocs = mp.cpu_count()

In [ ]:
p = mp.Pool(processes=nprocs, initializer=make_global, initargs=(no_buildings, done,))
p.map(driver, all_files)

# p = mp.Pool(processes=nprocs)
# p.map(driver, all_files)

# Delete raw_tif/ and images/ that have no buildings in them

In [9]:
with open('data/los_angeles/no_buildings_v2.txt', 'r') as f:
    no_buildings = f.readlines()
    no_buildings = [line.rstrip() for line in no_buildings]

In [10]:
fp = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512_v2/'

In [11]:
glob.glob(f'{fp}*')

['/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512_v2/raw_tif',
 '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512_v2/train']

In [12]:
for i in no_buildings:
    if os.path.exists(os.path.join(fp, 'train', 'images', f'{i}.npy')):
        os.remove(os.path.join(fp, 'train', 'images', f'{i}.npy'))
#     if os.path.exists(os.path.join(fp, 'raw_tif', f'{i}.tif')):
#         os.remove(os.path.join(fp, 'raw_tif', f'{i}.tif'))

# Find out which images are not 512x512

In [13]:
fp = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512_v2/'

In [13]:
total_count = len(os.listdir(f'{fp}train/images/'))
count = 0
for i in glob.glob(f'{fp}train/images/*'):
    npy = np.load(i)
    if npy.shape[0] != 512 or npy.shape[1] != 512:
        count += 1

print(count, total_count, count/total_count)

748 5931 0.1261170123082111


In [14]:
if not os.path.exists(os.path.join(fp, 'not_512')):
    os.mkdir(os.path.join(fp, 'not_512'))
    os.mkdir(os.path.join(fp, 'not_512', 'images'))
    os.mkdir(os.path.join(fp, 'not_512', 'masks'))

In [15]:
import shutil

In [16]:
for i in os.listdir(f'{fp}train/images/'):
    npy = np.load(f'{fp}train/images/{i}')
    if npy.shape[0] != 512 or npy.shape[1] != 512:
        shutil.move(os.path.join(fp, 'train', 'images', i), 
                    os.path.join(fp, 'not_512', 'images'))
        shutil.move(os.path.join(fp, 'train', 'masks', i.replace(".npy", "_mask.npy")), 
                    os.path.join(fp, 'not_512', 'masks'))

# Reshape non-512x512

In [18]:
import cv2
import matplotlib.pyplot as plt

dim = (512, 512)

for i in os.listdir(os.path.join(fp, 'not_512', 'images')):
    img = np.load(os.path.join(fp, 'not_512', 'images', i))
    mask = np.load(os.path.join(fp, 'not_512', 'masks', i.replace('.npy', '_mask.npy')))
    if img.shape[0] >= 250 and img.shape[1] >= 250:
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        resized_mask = cv2.resize(mask, dim, interpolation = cv2.INTER_AREA)
#         print(os.path.join(fp, 'train', 'images', i))
#         print(os.path.join(fp, 'train', 'masks', i.replace('.npy', '_mask.npy')))

        np.save(os.path.join(fp, 'train', 'images', i),
            resized)

        np.save(os.path.join(fp, 'train', 'masks', i.replace('.npy', '_mask.npy')),
            resized_mask)

#         plt.imshow(resized[:, :, :3])
#         plt.show()

#         plt.imshow(resized_mask)
#         plt.show()

In [21]:
len(os.listdir(os.path.join(fp, 'train', 'images')))

5365

# Split train-test-val

In [22]:
fp = '/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512_v2/'

In [28]:
from tqdm import tqdm

def train_val_split(root_dir, save_dir, train_percent):
    if not os.path.exists(save_dir):
        os.mkdir(os.path.join(save_dir))
        os.mkdir(os.path.join(save_dir, 'images'))
        os.mkdir(os.path.join(save_dir, 'masks'))
    
    all_images = list(os.listdir(os.path.join(root_dir, "images")))
    np.random.shuffle(all_images)

    num_train = int(len(all_images) * train_percent)
    val_images = all_images[num_train:]

    for img in tqdm(val_images):
        shutil.move(os.path.join(root_dir, "images", img), os.path.join(save_dir, "images"))
        shutil.move(os.path.join(root_dir, "masks", img.replace(".npy", "_mask.npy")), os.path.join(save_dir, "masks"))

In [29]:
train_val_split(os.path.join(fp, 'train'), os.path.join(fp, 'val'), 0.9)

100%|██████████| 537/537 [00:03<00:00, 176.31it/s]


In [30]:
train_val_split(os.path.join(fp, 'train'), os.path.join(fp, 'test'), 0.9)

100%|██████████| 483/483 [00:02<00:00, 193.13it/s]
